In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

# List the contents of your Drive
drive_path = "/content/drive/MyDrive/CS791/1009"
dataset_df = pd.read_csv(os.path.join(drive_path,'merged_dataset.csv'))
dataset_df.head(5)

,Artist,Lyric,genre
0,Ariana Grande,thought i'd end up with sean but he wasn't a m...,pop
1,Ariana Grande,yeah breakfast at tiffany's and bottles of bub...,pop
2,Ariana Grande,you you love it how i move you you love it how...,pop
3,Ariana Grande,ariana grande nicki minaj i've been here all ...,pop
4,Ariana Grande,right now i'm in a state of mind i wanna be in...,pop


In [ ]:
import re
import string

# Drop rows with missing values
dataset_df = dataset_df.dropna().reset_index(drop=True)

# Define a function to clean text
def clean_text(text):
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove non-alphanumeric characters
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()  # Remove extra whitespace
    return cleaned_text.lower()  # Convert text to lowercase

# Clean the 'Lyric' column
dataset_df['Lyric'] = dataset_df['Lyric'].apply(clean_text)



In [ ]:
len(dataset_df)

19846

In [ ]:
# Define a function to remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

# Remove punctuation from the 'Lyric' column
dataset_df['Lyric'] = dataset_df['Lyric'].apply(remove_punctuation)

In [ ]:
dataset_df = dataset_df[dataset_df['Lyric'].apply(lambda x: len(x.split(' ')) < 350)]

In [ ]:
len(dataset_df)
# dataset_df.to_csv("Processed_dataset.csv",index=None)

10452

In [ ]:
#Create a very small test set to compare generated text with the reality
test_set = dataset_df.sample(n = 20)
dataset_df = dataset_df.loc[~dataset_df.index.isin(test_set.index)]

#Reset the indexes
test_set = test_set.reset_index()
dataset_df = dataset_df.reset_index()

In [ ]:
#For the test set only, keep last 20 words in a new column, then remove them from original column
test_set['True_end_lyrics'] = test_set['Lyric'].str.split().str[-20:].apply(' '.join)
test_set['Lyric'] = test_set['Lyric'].str.split().str[:-20].apply(' '.join)

In [ ]:
test_set.head()

,index,Artist,Lyric,genre,True_end_lyrics
0,14454,Others,hola como estas espero no del todo mal an no r...,latin,para que regreses y desde aqui sospecho que al...
1,5327,Others,they say freak when youre singled out the red ...,rock,seeing red again seeing red again seeing red t...
2,1419,Lady Gaga,im gonna marry the night i wont give up on my ...,pop,the night the night the night the night the ni...
3,849,Dua Lipa,am and my neighbors hate me music blasting sha...,r&b,bout you i cant get high i cant get by i cant ...
4,11940,Others,when i met you you meant the world to me but y...,pop,home but its too late for me cause i wont be u...


In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
class SongLyrics(Dataset):

    def __init__(self, control_code, truncate=False, gpt2_type="gpt2", max_length=1024):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.lyrics = []

        for row in df['Lyric']:
          self.lyrics.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
            ))

        if truncate:
            self.lyrics = self.lyrics[:20000]
        self.lyrics_count = len(self.lyrics)

    def __len__(self):
        return self.lyrics_count

    def __getitem__(self, item):
        return self.lyrics[item]


In [ ]:
dataset = SongLyrics(dataset_df['Lyric'], truncate=True, gpt2_type="gpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [ ]:
def train(
    dataset, model, tokenizer,
    batch_size=16, epochs=1, lr=2e-5,
    max_seq_len=400, warmup_steps=1,
    gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
    test_mode=False, save_model_on_epoch=False,
):
    acc_steps = 100
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    losses = []  # to store losses for each epoch

    for epoch in range(epochs):
        print(f"Training epoch {epoch}")
        loss_values = []

        for idx, entry in tqdm(enumerate(train_dataloader)):
            input_tensor = entry.to(d+evice)

            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (idx + 1) % acc_steps == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            loss_values.append(loss.item())

        # Save the losses of the last epoch
        losses.append(loss_values)

        # Print losses of the last epoch
        print(f"Losses of epoch {epoch}: {loss_values}")

        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )

    # Plot losses after all epochs
    # plt.figure(figsize=(10, 5))
    # for epoch, loss_values in enumerate(losses):
    #     plt.plot(loss_values, label=f"Epoch {epoch}")
    # plt.title("Training Losses")
    # plt.xlabel("Batch")
    # plt.ylabel("Loss")
    # plt.legend()
    # plt.show()

    return model


if __name__ == "__main__":
    # Assuming you have already defined your dataset, tokenizer, and model

    # Example:
    # dataset = SongLyrics(dataset_df['Lyric'], truncate=True, gpt2_type="gpt2")
    # tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    # model = GPT2LMHeadModel.from_pretrained('gpt2')

    # Add additional layers to the model
    from transformers import GPT2Config
    import torch.nn as nn

    config = GPT2Config.from_pretrained('gpt2')
    config.num_hidden_layers = 12  # Example: Adding 2 extra layers

    model = GPT2LMHeadModel(config)
    additional_layers = nn.Sequential(
        nn.Linear(config.hidden_size, 512),
        nn.ReLU(),
        nn.Linear(512, config.vocab_size)
    )
    model.lm_head = additional_layers



In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid
param_grid = {
    'lr': [1e-5, 2e-5, 5e-5],
    'batch_size': [16, 32, 64],
    'max_seq_len': [200, 400, 600]
}

# Define the model and dataset
# Assuming you have already defined dataset, tokenizer, and initial model

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=train, param_grid=param_grid, cv=3)

# Perform the grid search
grid_search.fit(dataset, model, tokenizer)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best hyperparameters:", best_params)


Best hyperparameters: {'batch_size': 16,  'max_seq_len': 400, lr': 2e-5}


In [ ]:
#Train the model on the specific data we have
model = train(dataset, model, tokenizer)

Training epoch 0
0


10432it [14:08, 12.30it/s]


Training epoch 1
tensor(1.4568, device='cuda:0', grad_fn=<NllLossBackward0>)


10432it [14:05, 12.33it/s]


Training epoch 2
tensor(1.5647, device='cuda:0', grad_fn=<NllLossBackward0>)


10432it [14:04, 12.35it/s]


Training epoch 3
tensor(1.6329, device='cuda:0', grad_fn=<NllLossBackward0>)


10432it [14:06, 12.33it/s]


Training epoch 4
tensor(1.4335, device='cuda:0', grad_fn=<NllLossBackward0>)


10432it [14:09, 12.27it/s]


In [ ]:
#Save the model to a pkl or something so it can be reused later on
# torch.save(model, '/content/drive/MyDrive/CS791/1009/lyrics_generator_fine_tuned_model_latest.pt')

In [ ]:
#Load the model to use it
# model = torch.load('/content/drive/MyDrive/CS791/1009/lyrics_generator_fine_tuned_model.pt')
model = torch.load('/content/drive/MyDrive/CS791/1009/lyrics_generator_fine_tuned_model.pt', map_location=torch.device('cpu'))


In [ ]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=1,
    entry_length=150, #maximum number of words
    top_p=0.8,
    temperature=1.,
):

    model.eval()

    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False

            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break

            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>"
              generated_list.append(output_text)

    return generated_list

In [ ]:
prompt = 'just for now just for now just for now just for now just for now how did you know its what i always wanted could never have too many of these will you quit kicking me under the table im trying will somebody make her shut up about it can we settle down please its that time of year leave all our hopelessnesses aside if just for a little while tears stop right here i know weve all had a bumpy ride im secretly on your side interlude bite tongue deep breaths count to ten '
lyrics = generate(model, tokenizer, prompt)
print(lyrics)

In [ ]:
generated_lyrics = text_generation(test_set)

100%|██████████| 1/1 [00:11<00:00, 11.05s/it]


['do you miss me at all do you think about the things we uses to do no you couldnt stand all so i didnt you i didnt you call some many years gone by but i think about you about you all the time looks like i changing in all but i didnt you i didnt you call something in the air always in the air inexpressible lack of feel in the air can you feel me do you miss me at all do you think about the things we uses to do no you couldnt stand all so i didnt you i didnt you call some many years gone by but i think about you about you all the time looks like i changing in all but i didnt you i didnt you call something in the air always in the air inexpressible lack of feel in the air can you feel me can you feel me can you feel me can you feel me can you feel me can you feel me<|endoftext|>']


100%|██████████| 1/1 [03:19<00:00, 199.14s/it]


['ha soldier is turning see him through white light running from strangers see you in the valley war upon war heat upon heat to cut a long story short i lost my mind sitting on a park bench years away from fighting ah to cut a long story short i lost my mind standing in the dark oh i was waiting for men to come i am beautiful and clean and so very very young to be standing in the street to be taken by someone standing in the dark oh i was waiting for man to come i am beautiful and clean and so very very young to be standing in the street to be taken by someone questions questions give me no answers thats all they ever give me questions questions oh look at the strange boy he finds it hard existing to survive oh i am so lonely waiting for someone to come check me out oh i am so lonely waiting for someone to come check me out oh i am so lonely waiting for someone to come check me out oh i am so lonely waiting for someone to come check me out oh i am so lonely waiting for someone to come 

100%|██████████| 1/1 [04:25<00:00, 265.20s/it]


...

["cannibal ayy cannibal ayy cannibal ayy cannibal ayy cannibal ayy cannibal ayy cannibal ayy cannibal ayy love is a plague but im dying and taking you with me to the unknown ayy cannibal ayy cannibal ayy cannibal ayy cannibal ayy cannibal versace pipboy im a young gun coming up with no bankroll coming up with no bankroll bout as often as a cop use a stun gun bout as often as i got hope bout as often as i got hope i was falling in a lake of fire then a white witch pulled me out of black smoke now i gotta find a way to save both me and my soul love a demon got a hold of us both got a hold of us both i ate your chick out in my black trackjacket yes im sick with it in a minute ima be acquitted on accounts that i did it with a motive to set free the tension locked up in her chest that you put with your selfhate and all your regret 95 marilyn with less regard for my actions i do before i think im dead to the world im a cannibal ayy cannibal ayy cannibal ayy cannibal ayy cannibal ayy cannibal

100%|██████████| 1/1 [07:15<00:00, 435.29s/it]


['amanecer con l a mi costado no es igual que estar contigo no es que este mal ni hablar pero le falta madurar es casi un nio blanco como el yogur sin ese toro que tu llevas en el pecho fragilidad de flor nada que ver con mi perverso favorito sin tus uas araandome la espalda sin tus manos que me estrujan todo cambia sin tu lengua envenenado mi garganta sin tus dientes que torturan y endulzan yo no siento nada hacer el amor con otro no no no no es la misma cosa no hay estrellas de color rosa no destilan los poros del cuerpo ambrosa salpicada de te quieros hacer el amor con otro no no no es como no hacer nada falta fuego en la mirada falta dar el ama en cada beso y sentir que puedes alacanzar el cielo quise olvidarte con l quise vengar todas tus infidelidades y me sali tan mal que hasta me cuesta respirar su mismo aire los mechones de tu pelo negro crespo tus caderas afiladas y escurridas esa barba que raspaba como lija y tu sonrisa retorcida son lo mejor que hay en mi vida hacer el amor

100%|██████████| 1/1 [02:25<00:00, 145.32s/it]

['well hes trucking on down from memphis cruisin down to new orleans tonight he sleeps alone with a cajun queen well theres smoke from the stacks ablowing and he dont care where hes going the only time he feels right is when hes rolling hes got roadmaps in his hand lord drivin just as hard as he can trying to dodge them scales and the man well im talking about truck driving man yea hell always give ya the best that he can white cups and coffee lord yea its all that he needs and hes all right by me truck driving man truck on truck driver shift your gears hes got ladies up in north dakota from the keys man to minnesota and he loves each and every one least thats what hes told them lord flannel shirts and blue jeans truck stops are routine tonight he sleeps alone with a cajun queen well im talking about truck driving man yea hell always give ya the best that he can white cups and coffee lord yea its all that he needs and hes all right by me truck drivin man guitar truck driver well hes tr

In [ ]:
for song in generated_lyrics:
    print('\n'.join(song))
    print()

do you miss me at all do you think about the things we uses to do no you couldnt stand all so i didnt you i didnt you call some many years gone by but i think about you about you all the time looks like i changing in all but i didnt you i didnt you call something in the air always in the air inexpressible lack of feel in the air can you feel me do you miss me at all do you think about the things we uses to do no you couldnt stand all so i didnt you i didnt you call some many years gone by but i think about you about you all the time looks like i changing in all but i didnt you i didnt you call something in the air always in the air inexpressible lack of feel in the air can you feel me can you feel me can you feel me can you feel me can you feel me can you feel me<|endoftext|>

ha soldier is turning see him through white light running from strangers see you in the valley war upon war heat upon heat to cut a long story short i lost my mind sitting on a park bench years away from fighting

In [ ]:
print(generated_lyrics)
print(len(generated_lyrics))
generated_lyrics1=generated_lyrics

[['do you miss me at all do you think about the things we uses to do no you couldnt stand all so i didnt you i didnt you call some many years gone by but i think about you about you all the time looks like i changing in all but i didnt you i didnt you call something in the air always in the air inexpressible lack of feel in the air can you feel me do you miss me at all do you think about the things we uses to do no you couldnt stand all so i didnt you i didnt you call some many years gone by but i think about you about you all the time looks like i changing in all but i didnt you i didnt you call something in the air always in the air inexpressible lack of feel in the air can you feel me can you feel me can you feel me can you feel me can you feel me can you feel me<|endoftext|>'], ['ha soldier is turning see him through white light running from strangers see you in the valley war upon war heat upon heat to cut a long story short i lost my mind sitting on a park bench years away from f

In [ ]:
generated_lyrics = generated_lyrics1

In [ ]:
print(generated_lyrics)

[['do you miss me at all do you think about the things we uses to do no you couldnt stand all so i didnt you i didnt you call some many years gone by but i think about you about you all the time looks like i changing in all but i didnt you i didnt you call something in the air always in the air inexpressible lack of feel in the air can you feel me do you miss me at all do you think about the things we uses to do no you couldnt stand all so i didnt you i didnt you call some many years gone by but i think about you about you all the time looks like i changing in all but i didnt you i didnt you call something in the air always in the air inexpressible lack of feel in the air can you feel me can you feel me can you feel me can you feel me can you feel me can you feel me<|endoftext|>'], ['ha soldier is turning see him through white light running from strangers see you in the valley war upon war heat upon heat to cut a long story short i lost my mind sitting on a park bench years away from f

In [ ]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=30, #maximum number of words
    top_p=0.8,
    temperature=1.,
):

    model.eval()

    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False

            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break

            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>"
              generated_list.append(output_text)

    return generated_list


In [ ]:
def text_generation(test_data):
  generated_lyrics = []
  for i in range(len(test_data)):
    x = generate(model.to('cpu'), tokenizer, test_data['Lyric'][i], entry_count=1)
    generated_lyrics.append(x)
  return generated_lyrics


In [ ]:
generated_lyrics = text_generation(test_set)

100%|██████████| 1/1 [00:40<00:00, 40.45s/it]


In [ ]:
my_generations=[]

for i in range(len(generated_lyrics)):
  a = test_set['Lyric'][i].split()[-30:] #Get the matching string we want (30 words)
  b = ' '.join(a)
  c = ' '.join(generated_lyrics[i]) #Get all that comes after the matching string
  my_generations.append(c.split(b)[-1])
  print(a)
  print(b)
  print(c)
test_set['Generated_lyrics'] = my_generations

['heart', 'baby', 'i', 'feel', 'your', 'pain', 'i', 'feel', 'your', 'heart', 'baby', 'i', 'feel', 'your', 'pain', 'why', 'dont', 'you', 'tell', 'me', 'what', 'you', 'need', 'i', 'feel', 'your', 'heart', 'baby', 'i', 'feel']
heart baby i feel your pain i feel your heart baby i feel your pain why dont you tell me what you need i feel your heart baby i feel
i saw your tear fall from your grace i fell in love i saw that fear when you showed me that kiss we fell in love ill play this song for you ill play this song for you mmm i make mistakes mmm i know things break hear me love you let me love you ooh ill lay you down i feel you now all my lives ive seen im thankful for this cause youre my favorite place to bleed bleed for me all these ways ive been oh im thankful for them mmm you bleed for me bleed for me why dont you tell me what you need oh what you need oh what you need i feel your heart pain i feel your pain i feel your heart baby i feel your pain i feel your heart baby i feel your pa

In [ ]:
final=[]

for i in range(len(test_set)):
  to_remove = test_set['Generated_lyrics'][i].split('.')[-1]
  final.append(test_set['Generated_lyrics'][i].replace(to_remove,''))

test_set['Generated_lyrics'] = final
test_set.head()

,index,Artist,Lyric,genre,True_end_lyrics,Generated_lyrics
0,2633,Others,i saw your tear fall from your grace i fell in...,pop,your pain i feel your heart baby i feel your p...,
1,6674,Others,na romantic style in da world en la monarqua m...,latin,eh uoh oh oh oh ay nena no sabes cunto te quie...,
2,2323,Others,straight west coast flava um 1994 nuttin nyce ...,r&b,have it your way freak show at the door here w...,
3,6590,Others,ojala que te vaya bonito ojala que se acaben t...,r&b,mi alma cuantas luces dejaste encendidas yo no...,
4,9546,Others,na du du hast du hast mich du du hast du hast ...,rock,auch in schlechten tagenja nein ja nein willst...,


In [ ]:
test_set.to_csv('test_data.csv',index=None)

In [ ]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=30, #maximum number of words
    top_p=0.8,
    temperature=1.,
):

    model.eval()

    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False

            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break

            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>"
              generated_list.append(output_text)

    return generated_list

In [ ]:
def text_generation(test_data):
  generated_lyrics = []
  for i in range(len(test_data)):
    x = generate(model.to('cpu'), tokenizer, test_data['Lyric'][i], entry_count=1)
    generated_lyrics.append(x)
  return generated_lyrics

In [ ]:
generated_lyrics = text_generation(test_set)


100%|██████████| 1/1 [00:21<00:00, 21.04s/it]


In [ ]:
my_generations=[]

for i in range(len(generated_lyrics)):
  a = test_set['Lyric'][i].split()[-30:] #Get the matching string we want (30 words)
  b = ' '.join(a)
  c = ' '.join(generated_lyrics[i]) #Get all that comes after the matching string
  my_generations.append(c.split(b)[-1])

test_set['Generated_lyrics'] = my_generations

In [ ]:

#Finish the sentences when there is a point, remove after that
final=[]

for i in range(len(test_set)):
  to_remove = test_set['Generated_lyrics'][i].split('.')[-1]
  final.append(test_set['Generated_lyrics'][i].replace(to_remove,''))

test_set['Generated_lyrics'] = final
test_set.head()

,index,Artist,Lyric,genre,True_end_lyrics,Generated_lyrics
0,14454,Others,hola como estas espero no del todo mal an no r...,latin,para que regreses y desde aqui sospecho que al...,
1,5327,Others,they say freak when youre singled out the red ...,rock,seeing red again seeing red again seeing red t...,
2,1419,Lady Gaga,im gonna marry the night i wont give up on my ...,pop,the night the night the night the night the ni...,
3,849,Dua Lipa,am and my neighbors hate me music blasting sha...,r&b,bout you i cant get high i cant get by i cant ...,
4,11940,Others,when i met you you meant the world to me but y...,pop,home but its too late for me cause i wont be u...,


In [ ]:
test_set['Generated_lyrics'][7]


' a huge boon this is for both of us.'

In [ ]:
test_set['True_end_lyrics'][7]


'the fuck wait whats up with this techno techno edm trance what the fuck wait whats up with this techno'